In [2]:
import sys
import os

path = os.path.abspath('../..')
if path not in sys.path:
    sys.path.insert(0, path)
    
sys.path

from aqbt import AquariumBuildTools
aqtools = AquariumBuildTools.from_toml('creds.secret.toml')
aqtools.sessions

aq = aqtools.sessions['production']['aquarium']
aq.set_timeout(70)

In [33]:
fvs = aq.query({
    '__model__': 'FieldValue',
    '__query__': {
        'parent_class': 'Operation',
        'role': 'Input',
        'name': 'Genetic Material',
        'operation': {
            '__query__': {
                'user_id': 66,
                'status': {'__not__': 'planning'},
                'operation_type': {
                    '__query__': {
                        'name': 'Yeast Transformation'
                    }
                }
            }
        },
        '__return__': {
            'sample': [],
            'item': []
            'operation': []
        }
    },

})

fvs = [fv for fv in fvs if fv.operation.status in ['waiting', 'pending']]

group_by_samples = {}
for fv in fvs:
    group_by_samples.setdefault(fv.sample.id, list())
    group_by_samples[fv.sample.id].append(fv)

SyntaxError: invalid syntax (<ipython-input-33-8858bd1e66ec>, line 21)

In [43]:
def get_available_items(session, sample_id, sample_type):
    return session.query({
        '__model__': 'Item',
        '__query__': {
            'location': {'__not__': 'deleted'},
            'sample_id': sample_id,
            'sample_type': {
                '__query__': {
                    'name': sample_type
                }
            }
        }
    })

In [44]:
for sample_id, fvlist in group_by_samples.items():
    items = get_available_items(aq, sample_id, 'Fragment Stock')
    if len(fvlist) > len(items):
        print(aq.Sample.find(sample_id).name)
        print(len(items), ' < ', len(fvlist))
        print()

HIS-HIS<7XTetO-pMinCyc1-URGR-W8>
0  <  6

HIS-HIS<7XTetO-pMinCyc1-URGR-W34>
0  <  6

HIS-HIS<7XTetO-pMinCyc1-URGR-W5>
0  <  6

URA-URA<pGRR-W36W36-yeGFP>
0  <  1

URA-URA<pGRR-W17W34-yeGFP>
0  <  1

URA-URA<pGRR-W36W5-yeGFP>
0  <  1

URA-URA<pGRR-W17W17-yeGFP>
0  <  1

URA-URA<pGRR-W5W36-yeGFP>
0  <  1

pMOD6-pGRR-W5-RGR-W34
0  <  2

pMOD-LTR2-Bleo-pGRR-W20W8-RGR-W5
0  <  1

pMOD-LTR2-Bleo-pGRR-W20W8-RGR-W36
0  <  2

pMOD6-leu-pGRR-W5W34-RGR-W20
0  <  1

pMOD-HOKan-pGRR-W20-RGR-W8
0  <  1

pMOD-LTR1-NatMX-pGRR-W8-RGR-W10
0  <  1

pMOD6-pGRR-W19W5-RGR-W17
0  <  1

pMOD8-pGRR-W19W17-RGR-W5
0  <  2

pMOD6-pGRR-W5W19-RGR-W17
0  <  1

pMOD6-pGRR-W36-RGR-W8
0  <  1

pMOD8-pGALz4-yeGFP
0  <  1



In [29]:
pydent.aql.aql_schema

{'$schema': 'http://json-schema.org/draft-07/schema#',
 'title': 'Aquarium Query Language (AQL) Schema',
 'description': 'This schema validates query schema for AQL',
 'type': 'object',
 'definitions': {'time': {'type': 'object',
   'description': 'Delta time definition (from now)',
   'properties': {'__time__': {'type': 'object',
     'properties': {'__seconds__': {'type': 'integer'},
      '__hours__': {'type': 'integer'},
      '__days__': {'type': 'integer'},
      '__weeks__': {'type': 'integer'}},
     'minProperties': 1}},
   'minProperties': 1,
   'maxProperties': 1,
   'required': ['__time__'],
   'additionalProperties': False},
  'value': {'description': 'Valid query value',
   'anyOf': [{'type': 'string'},
    {'type': 'integer'},
    {'$ref': '#/definitions/time'}]},
  'valueArray': {'description': 'Valid query value array. Cannot be empty.',
   'type': 'array',
   'items': {'$ref': '#/definitions/value'},
   'minItems': 1},
  'sql': {'type': 'object',
   'description': 'Qu